In [1]:
import pandas as pd
import re
import csv

In [2]:
o = 'human'

df = pd.read_csv('input/Kinase_Substrate_Dataset EDITED', delimiter = '\t')
df = df[df.KIN_ORGANISM == df.SUB_ORGANISM]
df = df[df.KIN_ORGANISM == o]
df = df.loc[:, ['KIN_ORGANISM', 'SUB_ORGANISM', 'GENE', 'SUB_GENE', 'SUB_MOD_RSD']]
df['SUB_PHOS'] = df['SUB_GENE'] + '_' + df['SUB_MOD_RSD']
df.head()

,KIN_ORGANISM,SUB_ORGANISM,GENE,SUB_GENE,SUB_MOD_RSD,SUB_PHOS
6,human,human,EIF2AK1,EIF2S1,S52,EIF2S1_S52
7,human,human,EIF2AK1,EIF2S1,S49,EIF2S1_S49
10,human,human,PRKCD,HDAC5,S259,HDAC5_S259
11,human,human,PRKCD,PTPRA,S204,PTPRA_S204
12,human,human,PRKCD,HNRNPK,S302,HNRNPK_S302


In [3]:
#d = dict((o, [set(), set()]) for o in ['human', 'mouse', 'rat'])
d = {}
org = list(df.loc[:, 'KIN_ORGANISM'])
gene1 = list(df.loc[:, 'GENE'])
gene2 = list(df.loc[:, 'SUB_GENE'])
psites = list(df.loc[:, 'SUB_PHOS'])

for g1, g2, p in zip(gene1, gene2, psites):
    if g2 == g2 and p == p: # check if NaN
        if g1 not in d:
            d[g1] = [{p}, {g2}]
        else:
            d[g1][0].add(p)
            d[g1][1].add(g2)
            
print(len(d))


359


In [4]:
allwp = open('output-withphosphosite_phosphositeplus_all_{0}.gmt'.format(o), 'w+') # w phosphosite
fwp = open('output-withphosphosite_phosphositeplus_fourplusinteractions_{0}.gmt'.format(o), 'w+')
allnp = open('output_phosphositeplus_all_{0}.gmt'.format(o), 'w+')
fnp = open('output_phosphositeplus_fourplusinteractions_{0}.gmt'.format(o), 'w+')

uSubs = set()
numKSI = 0
numKins = 0
for k in d:
    # list of substrates for k
    temp = list(d[k][0]) # {sub_phos}
    temp1 = list(d[k][1]) # {sub}
    
    if len(d[k][1]) >= 4:
        # update counts
        numKSI += len(d[k][1])
        {uSubs.add(x) for x in d[k][1]}
        numKins += 1

        # add kinase info
        temp.insert(0, '{0}_phosphositeplus_{1}'.format(k, o))
        temp1.insert(0, '{0}_phosphositeplus_{1}'.format(k, o))

        # write to _four            
        #fwp.write('\t'.join(temp) + '\n')
        fnp.write('\t'.join(temp1) + '\n')

        # write to _all
        allwp.write('\t'.join(temp) + '\n')
        allnp.write('\t'.join(temp1) + '\n')

    else:
        temp.insert(0, '{0}_phosphositeplus_{1}'.format(k, o))        
        temp1.insert(0, '{0}_phosphositeplus_{1}'.format(k, o))

        allwp.write('\t'.join(temp) + '\n')
        allnp.write('\t'.join(temp1) + '\n')


print('{0}\n#kin: {1}\t#ksi: {2}\t#usubs: {3}'.format(o, numKins, numKSI, len(uSubs)))


allwp.close()
fwp.close()
allnp.close()
fnp.close()
#.write('\t'.join(list) + '\n')

human
#kin: 190	#ksi: 5379	#usubs: 2286
